In [86]:
import os
import json
from pprint import pprint

def count_json_files(directory):
    json_file_count = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(".json"):
                json_file_count += 1

    return json_file_count

train_directory = "/home/ssu35/lion/final/01.데이터/1.Training/"
train_json_count = count_json_files(train_directory)

val_directory = "/home/ssu35/lion/final/01.데이터/2.Validation/"
val_json_count = count_json_files(val_directory)

print(f"훈련 데이터 파일 개수: {train_json_count}")
print(f"검증 데이터 파일 개수: {val_json_count}")


훈련 데이터 파일 개수: 39591
검증 데이터 파일 개수: 5906


/home/ssu35/lion/final/01.데이터/1.Training

In [87]:
training_classes = os.listdir('/home/ssu35/lion/final/01.데이터/1.Training/라벨링데이터/')
print(training_classes)

['대안제시', '설명글', '주장', '글짓기', '찬성반대']


In [88]:
def get_unique_prompts(data):
    prompts = set()  # 중복을 허용하지 않는 집합(set)을 사용하여 중복 제거
    for item in data:
        prompts.add(item['info']['essay_prompt'])
    return list(prompts)

def read_json_files(directory):
    
    json_files = os.listdir(directory)
    json_datas = []

    for json_file in json_files:
        json_file_path = os.path.join(directory, json_file)
        with open(json_file_path) as f:
            json_data = json.load(f)
            json_datas.append(json_data)

    return json_datas


classes_paths = [os.path.join('/home/ssu35/lion/final/01.데이터/1.Training/라벨링데이터/', x) for x in training_classes]
class_datas = [read_json_files(directory) for directory in classes_paths]

for i, class_path in enumerate(classes_paths):
    num_files = count_json_files(class_path)
    class_datas = read_json_files(class_path)
    unique_prompts = get_unique_prompts(class_datas)
    print(f'{training_classes[i]} 클래스 - 프롬프트 개수: {len(unique_prompts)}, 파일 개수: {num_files}')


대안제시 클래스 - 프롬프트 개수: 7, 파일 개수: 5159
설명글 클래스 - 프롬프트 개수: 18, 파일 개수: 14575
주장 클래스 - 프롬프트 개수: 11, 파일 개수: 8596
글짓기 클래스 - 프롬프트 개수: 6, 파일 개수: 4574
찬성반대 클래스 - 프롬프트 개수: 8, 파일 개수: 6687


In [47]:
class_list = []
paragraphs_list = []
score_list = []

for class_path in classes_paths:
    for json_file in os.listdir(class_path):
        json_file_path = os.path.join(class_path, json_file)
        with open(json_file_path, 'r') as f:
            json_data = json.load(f)
            # essay_main_subject_list.append(json_data['info']['essay_main_subject'])
            class_list.append(class_path.split('/')[-1])
            text = ''
            for paragraph in json_data['paragraph']:
                text += paragraph['paragraph_txt'].replace('#@문장구분#', ' ').replace('\n', ' ')
            paragraphs_list.append(text)
            score = json_data['score']['essay_scoreT_avg'] + sum(s['paragraph_scoreT_avg'] for s in json_data['score']['paragraph_score']) / len(json_data['score']['paragraph_score'])
            score_list.append(score)


In [17]:
len(class_list), len(paragraphs_list), len(score_list)

(39591, 39591, 39591)

In [68]:
import pandas as pd

df = pd.DataFrame({'class': class_list, 'paragraphs': paragraphs_list, 'score': score_list})
df_sorted = df.sort_values(by='class').reset_index(drop=True)

In [69]:
df_sorted['class'].unique()

array(['글짓기', '대안제시', '설명글', '주장', '찬성반대'], dtype=object)

In [71]:
import csv
df_sorted.to_csv('/home/ssu35/lion/final/sorted_paragraphs.csv', index=False, quoting=csv.QUOTE_NONNUMERIC, encoding='utf-8')



PLM(KoichiYasuoka/roberta-base-korean-morph-upos)을 이용해 만든 vocab(output_file.pkl)을 불러오는 코드

In [57]:
import pickle
import pandas as pd
file_path='/home/ssu35/lion/final/vocab_by_plm.pkl'
with open(file_path, 'rb') as f:
    vocab_list = pickle.load(f)

In [58]:
df_sorted = pd.read_csv('/home/ssu35/lion/final/sorted_paragraphs.csv', encoding='utf-8')
df_vocab = pd.DataFrame({"vocab": vocab_list})
merged_df = pd.concat([df_sorted, df_vocab], axis=1)

In [59]:
merged_df

,class,paragraphs,score,vocab
0,글짓기,레오나르도 디세르 피에로 다 빈치 라는 이름을 가진 다빈치는 빈치 지역 출신의 세르...,22.481482,"[(레오나르도, NOUN), (디세르, NOUN), (피에로, NOUN), (다, ..."
1,글짓기,자연이란 자연적으로 생성되거나 갖추어진 지구상의 환경입니다. 자연은 항상 늘 사람...,30.111112,"[(자연, NOUN), (이란, ADP), (자연, NOUN), (적으로, CCON..."
2,글짓기,김만덕은 양인 집안에서 태어나 부모님이 어린시절부터 돌아가시고 김만덕은 은퇴한 기생...,30.661375,"[(김만덕, NOUN), (은, ADP), (양인 집안, NOUN), (에서, AD..."
3,글짓기,"나는 흰고래다. 벨루가 라고 불린다. 나는 하구역, 연안해역, 부빙역에 내 친구...",29.583333,"[(나, VERB), (는, ADP), (흰고래다, NOUN+AUX+PART), (..."
4,글짓기,최근 블랙홀의 모습을 사진으로 직접 찍어 화제가 되었던 적이 있습니다. 블랙홀을 관...,31.246032,"[(최근, ADV), (블랙홀, NOUN), (의, ADP), (모습, NOUN),..."
...,...,...,...,...
39586,찬성반대,저는 동물사육을 제한하는 것이 옳다고 생각합니다. 먼저 첫번째 이유는 다른 사람...,30.566666,"[(저, PRON), (는, ADP), (동물, NOUN), (사육, NOUN), ..."
39587,찬성반대,나는 우리나라의 모든 학생들에게 공교육비와 사교육비 할 것 없이 모든 교육을 제공해...,25.559260,"[(나, VERB), (는, ADP), (우리나라, NOUN), (의, ADP), ..."
39588,찬성반대,저는 우리나라 통일에 대해 찬성입니다. 왜냐하면 나라가 갈라져있어서 너무나 보고싶...,21.171430,"[(저, PRON), (는, ADP), (우리나라 통일, NOUN), (에, ADP..."
39589,찬성반대,저는 절약이 중요하다고는 생각하지만 필수적이라고는 생각하지 않습니다. 살짝 이기적...,30.407408,"[(저, PRON), (는, ADP), (절약, NOUN), (이, ADP), (중..."


In [60]:
for _ in vocab_list:
    print(_)
    break

[('레오나르도', 'NOUN'), ('디세르', 'NOUN'), ('피에로', 'NOUN'), ('다', 'ADV'), ('빈치', 'NOUN'), ('라는', 'PART'), ('이름', 'NOUN'), ('을', 'ADP'), ('가진', 'VERB+PART'), ('다빈치', 'NOUN'), ('는', 'ADP'), ('빈치 지역 출신', 'NOUN'), ('의', 'ADP'), ('세르', 'NOUN'), ('피에로', 'NOUN'), ('의', 'ADP'), ('아들', 'NOUN'), ('레오나르도', 'PROPN'), ('라는', 'PART'), ('뜻', 'NOUN'), ('을', 'ADP'), ('가졌', 'VERB+AUX+PART'), ('다', 'VERB+AUX+CCONJ'), ('다빈치', 'PUNCT'), ('는', 'ADP'), ('어릴', 'ADJ'), ('때무터 다각도', 'NOUN'), ('에서', 'ADP'), ('재능', 'NOUN'), ('을', 'ADP'), ('보였', 'VERB'), ('는데', 'VERB+AUX+CCONJ'), ('아버지', 'NOUN'), ('는', 'ADP'), ('그', 'DET'), ('재능', 'NOUN'), ('을', 'ADP'), ('알아보', 'VERB'), ('고', 'CCONJ'), ('피렌체', 'NOUN'), ('로', 'ADP'), ('이사를', 'NOUN'), ('가', 'ADP'), ('그림', 'NOUN'), ('을', 'ADP'), ('배우', 'VERB'), ('게했다', 'NOUN+PART+AUX+PART'), ('.', 'PUNCT'), ('30', 'NUM'), ('살이', 'NOUN'), ('된', 'VERB+PART'), ('다빈치', 'NOUN'), ('는', 'ADP'), ('밀라노', 'NOUN'), ('공국', 'NOUN'), ('에서', 'ADP'), ('활동', 'NOUN'), ('을', 'ADP'), ('하', 'VERB'), ('는데', 'CCO

In [61]:
from pprint import pprint
from collections import Counter
# POS 정보와 예시 단어를 저장할 딕셔너리
pos_info = {}

for vocab in vocab_list:
    for word, pos in vocab:
        if pos not in pos_info:
            pos_info[pos] = {'count': 1, 'examples': [word]}
        else:
            pos_info[pos]['count'] += 1
# count를 기준으로 내림차순 정렬
sorted_pos_info = dict(sorted(pos_info.items(), key=lambda item: item[1]['count'], reverse=True))

# 정렬된 결과 출력
pprint(sorted_pos_info)

{'ADJ': {'count': 283043, 'examples': ['어릴']},
 'ADJ+PART': {'count': 25600, 'examples': ['멋진']},
 'ADP': {'count': 1961037, 'examples': ['을']},
 'ADV': {'count': 446983, 'examples': ['다']},
 'AUX': {'count': 253165, 'examples': ['게']},
 'CCONJ': {'count': 795415, 'examples': ['고']},
 'DET': {'count': 69928, 'examples': ['그']},
 'NOUN': {'count': 2354786, 'examples': ['레오나르도']},
 'NOUN+AUX+CCONJ': {'count': 131, 'examples': ['라고']},
 'NOUN+AUX+PART': {'count': 118717, 'examples': ['만찬이다']},
 'NOUN+NOUN+AUX+PART': {'count': 13, 'examples': ['문']},
 'NOUN+PART+AUX+CCONJ': {'count': 47914, 'examples': ['지']},
 'NOUN+PART+AUX+PART': {'count': 79034, 'examples': ['게했다']},
 'NOUN+PART+CCONJ': {'count': 194521, 'examples': ['만들어낸다']},
 'NOUN+PART+PART': {'count': 279278, 'examples': ['유명한']},
 'NUM': {'count': 75130, 'examples': ['30']},
 'PART': {'count': 824325, 'examples': ['라는']},
 'PRON': {'count': 187950, 'examples': ['여기']},
 'PROPN': {'count': 13900, 'examples': ['레오나르도']},
 'PUNCT': 

In [62]:
# 주어진 POS 태그를 한국어로 변환하는 함수
def translate_upos(upos_string, upos_to_korean):
    translated_tags = []
    for tag in upos_string.split('+'):
        translated_tags.append(upos_to_korean.get(tag, tag))  # 태그가 딕셔너리에 없으면 원래 태그를 사용
    return '+'.join(translated_tags)

# 한국어로 변환하는 딕셔너리
upos_to_korean = {
    'ADJ': '형용사',
    'ADP': '전치사',
    'ADV': '부사',
    'AUX': '보조용언',
    'CCONJ': '접속사',
    'DET': '관형사',
    'INTJ': '감탄사',
    'NOUN': '명사',
    'NUM': '수사',
    'PART': '조사',
    'PRON': '대명사',
    'PROPN': '고유명사',
    'PUNCT': '구두점',
    'SCONJ': '종속 접속사',
    'SYM': '기호',
    'VERB': '동사'
}

# 한국어로 변환된 태그를 기준으로 새로운 딕셔너리 생성
translated_pos_info = {}

for pos, info in sorted_pos_info.items():
    translated_pos = translate_upos(pos, upos_to_korean)
    translated_pos_info[translated_pos] = {'count': f"{info['count']}개", 'examples': info['examples']}

pprint(translated_pos_info)

{'고유명사': {'count': '13900개', 'examples': ['레오나르도']},
 '관형사': {'count': '69928개', 'examples': ['그']},
 '구두점': {'count': '617169개', 'examples': ['다빈치']},
 '기호': {'count': '587개', 'examples': ['㎞']},
 '대명사': {'count': '187950개', 'examples': ['여기']},
 '동사': {'count': '818950개', 'examples': ['보였']},
 '동사+보조용언+접속사': {'count': '28887개', 'examples': ['다']},
 '동사+보조용언+조사': {'count': '91500개', 'examples': ['가졌']},
 '동사+접속사': {'count': '151710개', 'examples': ['받아']},
 '동사+접속사+보조용언+접속사': {'count': '12874개', 'examples': ['나가']},
 '동사+조사': {'count': '188345개', 'examples': ['가진']},
 '명사': {'count': '2354786개', 'examples': ['레오나르도']},
 '명사+명사+보조용언+조사': {'count': '13개', 'examples': ['문']},
 '명사+보조용언+접속사': {'count': '131개', 'examples': ['라고']},
 '명사+보조용언+조사': {'count': '118717개', 'examples': ['만찬이다']},
 '명사+조사+보조용언+접속사': {'count': '47914개', 'examples': ['지']},
 '명사+조사+보조용언+조사': {'count': '79034개', 'examples': ['게했다']},
 '명사+조사+접속사': {'count': '194521개', 'examples': ['만들어낸다']},
 '명사+조사+조사': {'count': '27

In [63]:
for key, value in translated_pos_info.items():
    count = value['count'].replace('개', '').replace(',', '')
    translated_pos_info[key]['count'] = int(count)

# count를 기준으로 내림차순 정렬
sorted_translated_pos_info = dict(sorted(translated_pos_info.items(), key=lambda x: x[1]['count'], reverse=True))

for key, value in sorted_translated_pos_info.items():
    sorted_translated_pos_info[key]['count'] = f"{value['count']}개"

sorted_translated_pos_info



{'명사': {'count': '2354786개', 'examples': ['레오나르도']},
 '전치사': {'count': '1961037개', 'examples': ['을']},
 '조사': {'count': '824325개', 'examples': ['라는']},
 '동사': {'count': '818950개', 'examples': ['보였']},
 '접속사': {'count': '795415개', 'examples': ['고']},
 '구두점': {'count': '617169개', 'examples': ['다빈치']},
 '부사': {'count': '446983개', 'examples': ['다']},
 '형용사': {'count': '283043개', 'examples': ['어릴']},
 '명사+조사+조사': {'count': '279278개', 'examples': ['유명한']},
 '보조용언': {'count': '253165개', 'examples': ['게']},
 '명사+조사+접속사': {'count': '194521개', 'examples': ['만들어낸다']},
 '동사+조사': {'count': '188345개', 'examples': ['가진']},
 '대명사': {'count': '187950개', 'examples': ['여기']},
 '동사+접속사': {'count': '151710개', 'examples': ['받아']},
 '명사+보조용언+조사': {'count': '118717개', 'examples': ['만찬이다']},
 '동사+보조용언+조사': {'count': '91500개', 'examples': ['가졌']},
 '명사+조사+보조용언+조사': {'count': '79034개', 'examples': ['게했다']},
 '수사': {'count': '75130개', 'examples': ['30']},
 '관형사': {'count': '69928개', 'examples': ['그']},
 '명사+조사+보조

In [64]:
batch_num = 10
for num, vocab in enumerate(vocab_list):
    for word, pos in vocab:
        if pos == 'NOUN+NOUN+AUX+PART':
            print(num)
            print(word)
            print(vocab)
            for i in range(len(vocab) // batch_num + 1):
                start = batch_num * i
                end = start + batch_num
                print(vocab[start:end])
            break
    if 'NOUN+NOUN+AUX+PART' in [pos for _, pos in vocab]:
        break


3236
문
[('바다', 'NOUN'), ('새물 이야기 문어', 'NOUN'), ('8', 'NUM'), ('개', 'ADP'), ('의 다리', 'NOUN'), ('누구', 'PRON'), ('일까요', 'AUX'), ('?', 'PUNCT'), ('오징어', 'NOUN'), ('야', 'ADP'), ('너', 'PRON'), ('니', 'ADP'), ('아니야 아니야', 'CCONJ'), ('나니', 'VERB+CCONJ'), ('다리가10', 'NOUN'), ('개', 'ADP'), ('야 다리', 'NOUN'), ('의', 'ADP'), ('빨판', 'NOUN'), ('이', 'ADP'), ('달렸', 'VERB'), ('어', 'CCONJ'), ('복', 'NOUN+PART+CCONJ'), ('어야', 'CCONJ'), ('너', 'AUX'), ('니', 'ADP'), ('아니야', 'VERB+CCONJ'), ('아니야', 'CCONJ'), ('나는', 'ADP'), ('빨판이 없어', 'NOUN'), ('나는', 'ADP'), ('다리', 'NOUN'), ('가운데', 'ADP'), ('옆', 'NOUN'), ('에', 'ADP'), ('입', 'NOUN'), ('이', 'ADP'), ('달려', 'VERB'), ('있', 'VERB+AUX+CCONJ'), ('어', 'PUNCT'), ('해파리', 'NOUN'), ('야', 'ADP'), ('너', 'NOUN'), ('니', 'ADP'), ('아니야 아니야', 'CCONJ'), ('나는', 'ADP'), ('입이', 'NOUN'), ('없', 'ADJ'), ('나', 'CCONJ'), ('는 바다의', 'NOUN'), ('카멜레온', 'NOUN'), ('이야', 'CCONJ'), ('문어', 'NOUN'), ('야', 'ADP'), ('너니', 'ADP'), ('맞아 맞아', 'VERB+CCONJ'), ('나', 'NOUN'), ('는', 'ADP'), ('다리', 'NOUN'), ('가', '

In [65]:
df_sorted.iloc[3236]['paragraphs']

'바다새물 이야기 문어 8개의 다리 누구 일까요?  오징어야 너니 아니야 아니야 나니 다리가10개야 다리의 빨판이 달렸어  복어야 너니 아니야 아니야 나는 빨판이 없어 나는 다리 가운데 옆에 입이 달려있어  해파리야 너니 아니야 아니야 나는 입이 없 나는 바다의 카멜레온 이야  문어야 너니 맞아 맞아 나는 다리가8개의 다리 가운데의 입이 있고 바다의 칼멜레온이라는 별명이 있어  그리고 나애 친구 중에는 독이 있는 문어가 있어  개의 이름은 파란고리문어라고해  그 녀석의 독은 치명 적이라서 조심해야하고 그리고 내 친구 중에는 몸의 형태를 바꾸는 친구가 있어  그리고 독이 있는 친구들이 있어  개네들은 다음에 이야기를 해줄게 그럼 안녕'

In [70]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,TokenClassificationPipeline
tokenizer=AutoTokenizer.from_pretrained("KoichiYasuoka/roberta-base-korean-morph-upos")
model=AutoModelForTokenClassification.from_pretrained("KoichiYasuoka/roberta-base-korean-morph-upos")
pipeline=TokenClassificationPipeline(tokenizer=tokenizer,model=model,aggregation_strategy="simple")

nlp = lambda x:[(x[t["start"]:t["end"]],t["entity_group"]) for t in pipeline(x)]

text = df_sorted.iloc[3236]['paragraphs']
nlp(str(text))


[('바다', 'NOUN'),
 ('새물 이야기 문어', 'NOUN'),
 ('8', 'NUM'),
 ('개', 'ADP'),
 ('의 다리', 'NOUN'),
 ('누구', 'PRON'),
 ('일까요', 'AUX'),
 ('?', 'PUNCT'),
 ('오징어', 'NOUN'),
 ('야', 'ADP'),
 ('너', 'PRON'),
 ('니', 'ADP'),
 ('아니야 아니야', 'CCONJ'),
 ('나니', 'VERB+CCONJ'),
 ('다리가10', 'NOUN'),
 ('개', 'ADP'),
 ('야 다리', 'NOUN'),
 ('의', 'ADP'),
 ('빨판', 'NOUN'),
 ('이', 'ADP'),
 ('달렸', 'VERB'),
 ('어', 'CCONJ'),
 ('복', 'NOUN+PART+CCONJ'),
 ('어야', 'CCONJ'),
 ('너', 'AUX'),
 ('니', 'ADP'),
 ('아니야', 'VERB+CCONJ'),
 ('아니야', 'CCONJ'),
 ('나는', 'ADP'),
 ('빨판이 없어', 'NOUN'),
 ('나는', 'ADP'),
 ('다리', 'NOUN'),
 ('가운데', 'ADP'),
 ('옆', 'NOUN'),
 ('에', 'ADP'),
 ('입', 'NOUN'),
 ('이', 'ADP'),
 ('달려', 'VERB'),
 ('있', 'VERB+AUX+CCONJ'),
 ('어', 'PUNCT'),
 ('해파리', 'NOUN'),
 ('야', 'ADP'),
 ('너', 'NOUN'),
 ('니', 'ADP'),
 ('아니야 아니야', 'CCONJ'),
 ('나는', 'ADP'),
 ('입이', 'NOUN'),
 ('없', 'ADJ'),
 ('나', 'CCONJ'),
 ('는 바다의', 'NOUN'),
 ('카멜레온', 'NOUN'),
 ('이야', 'CCONJ'),
 ('문어', 'NOUN'),
 ('야', 'ADP'),
 ('너니', 'ADP'),
 ('맞아 맞아', 'VERB+CCONJ'),
 ('나

In [72]:
nlp('개의 이름은 파란고리문어라고해  그 녀석의 독은 치명 적이라서')

[('개의 이름', 'NOUN'),
 ('은', 'ADP'),
 ('파란', 'ADJ'),
 ('고리문어', 'NOUN+AUX+PART'),
 ('라고', 'NOUN+AUX+CCONJ'),
 ('해', 'NOUN+PART+CCONJ'),
 ('그', 'DET'),
 ('녀석', 'NOUN'),
 ('의 독은', 'ADP'),
 ('치명', 'NOUN'),
 ('적', 'PART'),
 ('이', 'AUX'),
 ('라', 'ADP'),
 ('서', 'PUNCT')]

In [74]:
nlp("파란고리문어라고해")

[('파란', 'ADJ'),
 ('고리', 'NOUN'),
 ('문', 'NOUN+AUX+PART'),
 ('어라고', 'NOUN+AUX+CCONJ'),
 ('해', 'NOUN+PART+CCONJ')]

In [94]:
sorted_df = merged_df.sort_values(by='score', ascending=False)
sorted_df

,class,paragraphs,score,vocab
31119,주장,우선 전제를 잡고 가자면 저는 e스포츠의 정식 스포츠화에 대하여 찬성하는 입장이라고...,33.0,"[(우선, ADV), (전제, NOUN), (를, ADP), (잡, VERB), (..."
1509,글짓기,"""내게 신이란 우주만물에 대한 경외감이다."" 라는 천재 물리학자 아인슈타인의 말이 ...",33.0,"[("", PUNCT), (내게, ADV), (신, NOUN), (이, AUX), (..."
11858,설명글,내가 생각하는 예절의 기준은 상대방을 향한 존중이다. 존중은 다른 사람의 의견을 ...,33.0,"[(내, PRON), (가, ADP), (생각, NOUN), (하는, PART), ..."
5210,대안제시,요즘 중국이 우리나라 문화를 자신의 문화라며 우기고 있습니다. 김치를 포함한 한복...,33.0,"[(요즘, ADV), (중국, NOUN), (이, ADP), (우리나라 문화, NO..."
5211,대안제시,반려동물을 키울때 그럼 동의서나 아님 강아지 고양이등 반려동물 캠페인을 만드는것 입...,33.0,"[(반려동물, NOUN), (을, ADP), (키울, VERB), (때, NOUN)..."
...,...,...,...,...
7397,대안제시,현대인들은 강아지나 고양이 등 다양한 반려동물을 키우며 행복과 위로를 받고 있습니다...,0.0,"[(현대인, NOUN), (들, PART), (은, ADP), (강아지, NOUN)..."
39216,찬성반대,민주주의 사회에서의 의사결정은 다수결의 원칙으로 정합니다. 대통령이나 국회의원 선...,0.0,"[(민주주의 사회, NOUN), (에서의, ADP), (의사, NOUN), (결정,..."
19512,설명글,여러분은 진정한친구에대해 생각해본 적이 있나요? 집에 항상 같이 비밀을 마음껏 얘...,0.0,"[(여러분, PRON), (은, ADP), (진정한, VERB+PART), (친구,..."
31207,주장,현대 과학은 나날이 발전해 나아가고 있습니다. 여기서 '현대 과학' 이란 현재 여...,0.0,"[(현대 과학, NOUN), (은, ADP), (나날이, ADV), (발전해, NO..."


In [107]:
wrt_df = sorted_df[(sorted_df['score'] > 30) & (sorted_df['class'] == '글짓기')]


In [117]:
# 한국어로 변환하는 딕셔너리를 전역으로 설정
upos_to_korean = {
    'ADJ': '형용사',
    'ADP': '전치사',
    'ADV': '부사',
    'AUX': '보조용언',
    'CCONJ': '접속사',
    'DET': '관형사',
    'INTJ': '감탄사',
    'NOUN': '명사',
    'NUM': '수사',
    'PART': '조사',
    'PRON': '대명사',
    'PROPN': '고유명사',
    'PUNCT': '구두점',
    'SCONJ': '종속 접속사',
    'SYM': '기호',
    'VERB': '동사'
}

def translate_upos(upos_string):
    return '+'.join([upos_to_korean.get(tag, tag) for tag in upos_string.split('+')])

def sort_pos_info(target_df):
    pos_info = {}
    # 품사 정보 추출
    for vocab in target_df['vocab']:
        for word, pos in vocab:
            if pos not in pos_info:
                pos_info[pos] = {'count': 1, 'examples': [word]}
            else:
                pos_info[pos]['count'] += 1
    # 내림차순으로 품사 정보 정렬
    sorted_pos_info = {pos: info for pos, info in sorted(pos_info.items(), 
                            key=lambda item: item[1]['count'], reverse=True)}
    # 품사 태그를 한국어로 변환
    translated_pos_info = {translate_upos(pos): info for pos, info in sorted_pos_info.items()}
    # 정수로 변환된 count로 재정렬
    sorted_translated_pos_info = {pos: {'count': f"{info['count']}개", 'examples': info['examples']}
                                  for pos, info in sorted(translated_pos_info.items(), 
                                  key=lambda item: int(item[1]['count']), reverse=True)}

    return sorted_translated_pos_info

wrt_info = sort_pos_info(wrt_df)


In [120]:
def filter_specific_pos(target_df, specific_pos):
    filtered_pos_info = {}
    for vocab in target_df:
        for word, pos in vocab:
            if pos == specific_pos:
                if pos not in filtered_pos_info:
                    filtered_pos_info[pos] = {'count': 1, 'examples': [word]}
                else:
                    filtered_pos_info[pos]['count'] += 1

    return filtered_pos_info

# 이 함수는 target_df 데이터프레임과 필터링하려는 특정 품사를 인자로 받아 해당 품사의 정보를 반환합니다.
# 사용 예:
# specific_pos_info = filter_specific_pos(target_df, 'NOUN') # 'NOUN' 품사만 추출
# print(specific_pos_info)


['신',
 '우주',
 '만물',
 '천재 물리학자 아인슈타인',
 '말',
 '선',
 '말',
 '때',
 '생각',
 '세상',
 '신',
 '세계',
 '우주',
 '신',
 '인간',
 '증명',
 '믿음',
 '허상',
 '뿐',
 '우주',
 '두려움',
 '수',
 '신',
 '가정',
 '신',
 '우주',
 '호기심',
 '인간',
 '게',
 '관심',
 '의문',
 '호기심',
 '것',
 '블랙홀',
 '물리학자 아인슈타인',
 '블랙홀',
 '존재',
 '자신',
 '상대',
 '이론',
 '가설',
 '약',
 '년 뒤',
 '과학자',
 '개의 대륙',
 '대',
 '전파',
 '망원경',
 '원자시계로 시간',
 '사건',
 '지평선 망원경',
 '공동 연구',
 '지구',
 '자전',
 '전파',
 '망원경',
 '위치',
 '용량',
 '페타바이트',
 '고속',
 '웹',
 '년',
 '자료',
 '아인슈타인',
 '이론',
 '블랙홀',
 '존재',
 '우주',
 '미스터리',
 '관측',
 '블랙홀',
 '빛',
 '만년',
 '현재',
 '관측',
 '우주',
 '반경',
 '약',
 '광년',
 '우주',
 '먼지',
 '신경',
 '하늘',
 '우주',
 '별',
 '별',
 '생각',
 '뿐',
 '러',
 '행성',
 '외계인',
 '행성',
 '우주',
 '정도',
 '상상',
 '수',
 '정도',
 '태양계',
 '모래알',
 '우주',
 '해수욕장',
 '정도',
 '우주',
 '년',
 '지구',
 '우주',
 '중심',
 '태양',
 '천체',
 '지구',
 '중심',
 '지구',
 '태양',
 '행성',
 '사실',
 '천문 학자',
 '지구',
 '환경',
 '생명체',
 '진화',
 '것',
 '수',
 '과학',
 '증거',
 '최근',
 '년',
 '행성',
 '존재',
 '행성',
 '사실',
 '태양계',
 '우주',
 '유일',
 '존재',
 '행성',
 '증명',
 '결론',
 '생명